In [34]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import joblib

LEFT = 0
RIGHT = 1

articles = pd.read_csv('binary_bias.csv')

#randomizes rows
articles = articles.sample(frac=1)

stemmer = SnowballStemmer('english')
words = stopwords.words("english")

articles['cleaned'] = articles['article'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

X_train, X_test, y_train, y_test = train_test_split(articles['cleaned'], articles['bias'], test_size=0.2)

### CLASSIFIERS ###

# RidgeClassifier
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('best',  SelectKBest(k=10000)),
                     ('model', RidgeClassifier())])
rc_model = pipeline.fit(X_train, y_train)
joblib.dump(rc_model, 'rc_model.joblib')

# LinearSVC
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('best',  SelectKBest(k=10000)),
                     ('model', LinearSVC(C=1.0, penalty='l2', max_iter=6000, dual=False))])
lsvc_model = pipeline.fit(X_train, y_train)
joblib.dump(lsvc_model, 'lsvc_model.joblib')

# MultinomialNB
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('best',  SelectKBest(k=10000)),
                     ('model', MultinomialNB())])
mnb_model = pipeline.fit(X_train, y_train)
joblib.dump(mnb_model, 'mnb_model.joblib')

# BernoulliNB
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('best',  SelectKBest(k=10000)),
                     ('model', BernoulliNB())])
bnb_model = pipeline.fit(X_train, y_train)
joblib.dump(bnb_model, 'bnb_model.joblib')

# LogisticRegression
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('best',  SelectKBest(k=10000)),
                     ('model', LogisticRegression())])
lr_model = pipeline.fit(X_train, y_train)
joblib.dump(lr_model, 'lr_model.joblib')

# SGDClassifier
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 2), stop_words="english")),
                     ('best',  SelectKBest(k=10000)),
                     ('model', SGDClassifier())])
sgd_model = pipeline.fit(X_train, y_train)
joblib.dump(sgd_model, 'sgd_model.joblib')

### ACCURACY ###

vectorizer = lsvc_model.named_steps['vect']
best = lsvc_model.named_steps['best']

rc = rc_model.named_steps['model']
lsvc = lsvc_model.named_steps['model']
mnb = mnb_model.named_steps['model']
bnb = bnb_model.named_steps['model']
lr = lr_model.named_steps['model']
sgd = sgd_model.named_steps['model']

feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in best.get_support(indices=True)]
feature_names = np.asarray(feature_names)

target_names = ['Left', 'Right']

print("\nRC Accuracy: " + str(round(rc_model.score(X_test, y_test)*100, 2)) + "%")
print("\nLSVC Accuracy: " + str(round(lsvc_model.score(X_test, y_test)*100, 2)) + "%")
print("\nMNB Accuracy: " + str(round(mnb_model.score(X_test, y_test)*100, 2)) + "%")
print("\nBNB Accuracy: " + str(round(bnb_model.score(X_test, y_test)*100, 2)) + "%")
print("\nLR Accuracy: " + str(round(lr_model.score(X_test, y_test)*100, 2)) + "%")
print("\nSGD Accuracy: " + str(round(sgd_model.score(X_test, y_test)*100, 2)) + "%")

print("\n\nTop 10 keywords for bias:\n-------------------------")
rc_top = np.argsort(rc.coef_[0])[-10:]
lsvc_top = np.argsort(lsvc.coef_[0])[-10:]
mnb_top = np.argsort(mnb.coef_[0])[-10:]
bnb_top = np.argsort(bnb.coef_[0])[-10:]
lr_top = np.argsort(lr.coef_[0])[-10:]
sgd_top = np.argsort(sgd.coef_[0])[-10:]

print("\nRC: %s" % (" ".join(feature_names[rc_top])))
print("\nLSVC: %s" % (" ".join(feature_names[lsvc_top])))
print("\nMNB: %s" % (" ".join(feature_names[mnb_top])))
print("\nBNB: %s" % (" ".join(feature_names[bnb_top])))
print("\nLR: %s" % (" ".join(feature_names[lr_top])))
print("\nSGD: %s" % (" ".join(feature_names[sgd_top])))




RC Accuracy: 70.54%

LSVC Accuracy: 71.09%

MNB Accuracy: 65.42%

BNB Accuracy: 69.71%

LR Accuracy: 69.57%

SGD Accuracy: 69.02%


Top 10 keywords for bias:
-------------------------

RC: document administr arm media alleg weiner presid tax obama obamacar

LSVC: presid especi document use left tax alleg arm obama obamacar

MNB: year white administr tax gun ir state republican presid obama

BNB: nation report polit republican peopl new state year obama presid

LR: report benghazi media tax weiner administr presid ir obamacar obama

SGD: premium told media presid bad tax especi use obama obamacar
